In [1]:
import cv2
#유튜브 영상 받기 위한
import pafy
url = "https://www.youtube.com/watch?v=4EKo44DUvjg"
video = pafy.new(url)
best = video.getbest(preftype="mp4")

In [2]:
#ml5 사전설정
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import time
import argparse #사용 안함
import posenet
import easydict
args = easydict.EasyDict({
    "model":101,
    "cam_id":"test_video.mp4",
    "cam_width":10,
    "cam_height":10,
    "scale_factor":0.7125,
    "file":None
})

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
def posenet_search():
    with tf.Session() as sess:
        model_cfg, model_outputs = posenet.load_model(args.model, sess)
        output_stride = model_cfg['output_stride']

        if args.file is not None:
            cap = cv2.VideoCapture(args.file)
        else:
            cap = cv2.VideoCapture(args.cam_id)
        cap.set(3, args.cam_width)
        cap.set(4, args.cam_height)

        start = time.time()
        frame_count = 0
        while True:
            input_image, display_image, output_scale = posenet.read_cap(
                cap, scale_factor=args.scale_factor, output_stride=output_stride)

            heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = sess.run(
                model_outputs,
                feed_dict={'image:0': input_image}
            )

            pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multi.decode_multiple_poses(
                heatmaps_result.squeeze(axis=0),
                offsets_result.squeeze(axis=0),
                displacement_fwd_result.squeeze(axis=0),
                displacement_bwd_result.squeeze(axis=0),
                output_stride=output_stride,
                max_pose_detections=10,
                min_pose_score=0.15)

            keypoint_coords *= output_scale

            # TODO this isn't particularly fast, use GL for drawing and display someday...
            overlay_image,cv_key_points = posenet.draw_skel_and_kp(
                display_image, pose_scores, keypoint_scores, keypoint_coords,
                min_pose_score=0.15, min_part_score=0.1)

            cv2.imshow('posenet', overlay_image)
            frame_count += 1
            if cv2.waitKey(1) & 0xFF == ord('q'):
                cap.release()
                cv2.destroyAllWindows()
                break

        print('Average FPS: ', frame_count / (time.time() - start))

In [4]:
#openpose coco 사전설정
BODY_PARTS = {0: "Nose", 1: "Neck", 2: "RShoulder", 3: "RElbow", 4: "RWrist",
                   5: "LShoulder", 6: "LElbow", 7: "LWrist", 8: "RHip", 9: "RKnee",
                   10: "RAnkle", 11: "LHip", 12: "LKnee", 13: "LAnkle", 14: "REye",
                   15: "LEye", 16: "REar", 17: "LEar", 18: "Background"}

POSE_PAIRS = [[0, 1], [0, 14], [0, 15], [1, 2], [1, 5], [1, 8], [1, 11], [2, 3], [3, 4],
                   [5, 6], [6, 7], [8, 9], [9, 10], [12, 13], [11, 12], [14, 16], [15, 17]]
proto_file = "../openpose-master/models/pose/coco/pose_deploy_linevec.prototxt"
weights_file = "../openpose-master/models/pose/coco/pose_iter_440000.caffemodel"
net = cv2.dnn.readNetFromCaffe(proto_file, weights_file)
image_height = 368
image_width = 368

In [5]:
def openpose():
    if args.file is not None:
        cap = cv2.VideoCapture(args.file)
    else:
        cap = cv2.VideoCapture(args.cam_id)
    cap.set(3, args.cam_width)
    cap.set(4, args.cam_height)
    threshold=0.2
    start = time.time()
    frame_count = 0
    
    while True:
        res, frame = cap.read()
        input_blob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (image_width, image_height), (0, 0, 0), swapRB=False, crop=False)
        points = []
        net.setInput(input_blob)

        # 결과 받아오기
        out = net.forward()
        out_height = out.shape[2]
        out_width = out.shape[3]

        # 원본 이미지의 높이, 너비를 받아오기
        frame_height, frame_width = frame.shape[:2]

        for i in range(len(BODY_PARTS)):

            # 신체 부위의 confidence map
            prob_map = out[0, i, :, :]

            # 최소값, 최대값, 최소값 위치, 최대값 위치
            min_val, prob, min_loc, point = cv2.minMaxLoc(prob_map)

            # 원본 이미지에 맞게 포인트 위치 조정
            x = (frame_width * point[0]) / out_width
            x = int(x)
            y = (frame_height * point[1]) / out_height
            y = int(y)

            if prob > threshold:  # [pointed]
                cv2.circle(frame, (x, y), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
                cv2.putText(frame, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 1, lineType=cv2.LINE_AA)

                points.append((x, y))
            else :
                points.append(None)
        

        for pair in POSE_PAIRS:
            part_a = pair[0]  # 0 (Head)
            part_b = pair[1]  # 1 (Neck)
            if points[part_a] and points[part_b]:
                cv2.line(frame, points[part_a], points[part_b], (0, 255, 0), 3)

        cv2.imshow('posenet', frame)
        frame_count += 1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break

    print('Average FPS: ', frame_count / (time.time() - start))

In [14]:
if __name__ == "__main__":
    openpose()

Average FPS:  0.7649156782907999


In [4]:
if __name__ == "__main__":
    posenet_search()

C:\Users\COM\DLP\posenet_python\posenet\converter\config.py:9: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = yaml.load(cfg_f)


Average FPS:  18.906395273369398


TypeError: 'cv2.VideoCapture' object is not subscriptable